In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
from scipy.spatial import KDTree
from nltk import edit_distance


Loading data

In [ ]:
path = "/kaggle/input/foursquare-location-matching/"

pairs_df = pd.read_csv(path + "pairs.csv")
train_df = pd.read_csv(path + "train.csv")
test_df = pd.read_csv(path + "test.csv")
sample_submission = pd.read_csv(path + "sample_submission.csv")

In [ ]:
train_df.head()

In [ ]:
t1 = pairs_df[pairs_df.address_1 == 'Terminal 1']

In [ ]:
t1.iloc[10:15].T

In [ ]:
# vectorized haversine function
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    """
    slightly modified version: of http://stackoverflow.com/a/29546836/2901002

    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees or in radians)

    All (lat, lon) coordinates must have numeric dtypes and be of equal length.

    """
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))

In [ ]:
# rounding = 0

def clean_name(name_col):
    return name_col.str.lower()\
                    .str.replace(",", "")\
                    .str.replace(".", "")\
                    .str.replace("'", "")\
                    .str.replace("the ", "")\
                    .str.split(" ")


def name_dist(token_1, token_2):
    return edit_distance(token_1, token_2, transpositions=True) / max(len(token_1), len(token_2))


In [ ]:
rounding = 0

def generate_matches(df):
    
    lats = df['latitude'].to_list()
    lons = df['longitude'].to_list()
    Z = tuple(zip(lats, lons))
    tree = KDTree(Z)
    loc_data = tree.query(Z, 3, p = 1, workers = -1)  
    
    df.loc[:,"clean_name"] = clean_name(df.categories.astype(str))

    matches = {
        a : {a} for a in df.id.values
    }
        
    for i, row_1 in df.iterrows():
        indx = loc_data[1][i]
        
        lat_1 = row_1.latitude
        clean_name_1 = row_1.clean_name
        
        for j in indx:
            row_2 = df.iloc[j, :]
            if haversine(lat_1, row_2.latitude, row_1.longitude, row_2.longitude) < 1.0:
                if name_dist(clean_name_1, row_2.clean_name) < 0.9:
                    matches[row_1.id].update(matches[row_2.id])
                    matches[row_2.id].update(matches[row_1.id])

    for _id, val in matches.items():

        matches[_id] = " ".join(list(val))
        
    return pd.DataFrame({
                "id" : list(matches.keys()),
                "matches" : list(matches.values())
            })

In [ ]:
submission = generate_matches(test_df)

In [ ]:
submission

In [ ]:
submission.to_csv("submission.csv", index=False)